# Data Overview & Cleaning

- Loads the synthetic sessions and previews core stats.
- Drops rows missing QoE KPIs (`page_load_time_ms`, `startup_delay_ms`, `buffering_ratio`).
- Standardizes types for categories, IDs, and numerics.
- Reports missingness, memory usage, and preserves relevant radio/context features.
- Outputs: cleaned table as `synthetic_qoe_sessions_clean.csv` (plus optional `.parquet`).


In [ ]:
# Step 1: Imports
import pandas as pd
import numpy as np

# Load the synthetic QoE session dataset
df = pd.read_csv("synthetic_qoe_sessions.csv", parse_dates=['timestamp'])

# Step 2: Display dataset shape (rows, columns)
print(f"Dataset shape: {df.shape}")  # Rows, Columns

# Step 3: Preview the first and last 5 rows to spot structure or anomalies
display(df.head())
display(df.tail())

# Step 4: Check for missing (NaN) values in each column
nan_counts = df.isnull().sum()
print("NaN counts per column:")
print(nan_counts)

# Step 5: Show % of missing values (for context)
print("\n% Missing values per column:")
print((nan_counts / len(df) * 100).round(2))

# Step 6: Display any rows containing NaN values for inspection (sample)
nan_rows = df[df.isnull().any(axis=1)]
print(f"Rows with NaNs: {len(nan_rows)}")
display(nan_rows.head(10))


In [ ]:
# Step 7: Light data hygiene — enforce valid ranges
# Timestamp: ensure tz-aware (UTC)
if getattr(df['timestamp'].dt, 'tz', None) is None:
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC')

# De-duplicate session_id if any
if 'session_id' in df.columns:
    dup_count = df.duplicated('session_id').sum()
    if dup_count:
        print(f"Found {dup_count} duplicate session_id rows; keeping first occurrence.")
        df = df.drop_duplicates('session_id', keep='first')

# Valid ranges for QoE (buffering already clipped in generator, but enforce)
df.loc[(df['buffering_ratio'] < 0) | (df['buffering_ratio'] > 0.20), 'buffering_ratio'] = np.nan

# Valid ranges for radio KPIs (only where present; keep NaN for Wi-Fi)
if {'rsrp_dbm','rsrq_db','sinr_db'}.issubset(df.columns):
    df.loc[(df['rsrp_dbm'] < -140) | (df['rsrp_dbm'] > -40), 'rsrp_dbm'] = np.nan
    df.loc[(df['rsrq_db']  < -20)  | (df['rsrq_db']  > -3),  'rsrq_db']  = np.nan
    df.loc[(df['sinr_db']  < -5)   | (df['sinr_db']  > 40),  'sinr_db']  = np.nan

print("Completed basic range checks and timestamp localization.")


In [ ]:
# Step 8: Selective cleaning policy
core_qoe = ['page_load_time_ms', 'startup_delay_ms', 'buffering_ratio']
before = df.shape[0]
df_clean = df.dropna(subset=core_qoe).copy()
after = df_clean.shape[0]
print(f"Dropped {before - after} rows missing core QoE metrics. Shape now: {df_clean.shape}")

# Step 9: Dtype optimization via a single astype() map
astype_map = {}

# Categoricals
for col in ['country', 'device', 'network_type', 'operator', 'band']:
    if col in df_clean.columns:
        astype_map[col] = 'category'

# Nullable integer IDs
for col in ['pci', 'tac', 'cell_id', 'channel_number']:
    if col in df_clean.columns:
        astype_map[col] = 'Int64'

# QoE metrics
astype_map.update({
    'page_load_time_ms': 'int32',
    'startup_delay_ms': 'int32',
    'buffering_ratio': 'float32',
})

# Radio KPIs
for col in ['rsrp_dbm', 'rsrq_db', 'sinr_db']:
    if col in df_clean.columns:
        astype_map[col] = 'float32'

# Apply in one go (no chained assignment)
df_clean = df_clean.astype(astype_map, copy=False)

# Step 10: Basic sanity summaries
print("\nPost-clean dtypes:")
print(df_clean.dtypes)

mem_mb = df_clean.memory_usage(deep=True).sum() / 1e6
print(f"\nEstimated memory footprint: {mem_mb:.2f} MB")

print("\nMissingness after cleaning (should be 0 for core QoE):")
print(df_clean[core_qoe].isnull().sum())

display(df_clean.head())


In [ ]:
# Step 11: Save the cleaned dataset for the next analysis steps
# CSV (existing behavior)
df_clean.to_csv("synthetic_qoe_sessions_clean.csv", index=False)

# Parquet
try:
    df_clean.to_parquet("synthetic_qoe_sessions_clean.parquet", index=False)
    print("Saved: synthetic_qoe_sessions_clean.csv and synthetic_qoe_sessions_clean.parquet")
except Exception as e:
    print("Parquet save skipped (install pyarrow or fastparquet to enable). CSV saved.")
